### Bike Sharing Station Data Collection & Preperation
In this notebook, we will download data on the bicycle sharing stations that are referenced by the our trip data.
The station data is publicly available as GeoJSON [here](http://bikeshare.metro.net/stations/json/).

In [1]:
import urllib.request
import json
import pandas as pd

In [2]:
# emulate a real browser by using common headers
# without these headers metro bike share blocks requests with a 403 forbidden error
# be nice to metro bike share and run this cell only ones
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

req = urllib.request.Request('http://bikeshare.metro.net/stations/json/', headers=headers)

with urllib.request.urlopen(req) as url:
	stations_data_json = json.loads(url.read().decode())

In [3]:
stations_df = pd.json_normalize(stations_data_json['features'])
stations_df.head(2)

,type,geometry.coordinates,geometry.type,properties.addressStreet,properties.addressCity,properties.addressState,properties.addressZipCode,properties.bikesAvailable,properties.closeTime,properties.docksAvailable,...,properties.kioskConnectionStatus,properties.kioskType,properties.latitude,properties.longitude,properties.hasGeofence,properties.classicBikesAvailable,properties.smartBikesAvailable,properties.electricBikesAvailable,properties.isArchived,properties.clientVersion
0,Feature,"[-118.25854, 34.0485]",Point,700 Flower St,DTLA,CA,90017,2,05:39:00,28,...,Active,1,34.04850,-118.25854,False,2,0,0,False,5.2.2.26
1,Feature,"[-118.25667, 34.04554]",Point,729 S Olive Street,DTLA,CA,90014,9,05:39:00,22,...,Active,1,34.04554,-118.25667,False,9,0,0,False,5.2.2.26


In [4]:
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   type                               225 non-null    object 
 1   geometry.coordinates               225 non-null    object 
 2   geometry.type                      225 non-null    object 
 3   properties.addressStreet           225 non-null    object 
 4   properties.addressCity             225 non-null    object 
 5   properties.addressState            225 non-null    object 
 6   properties.addressZipCode          225 non-null    object 
 7   properties.bikesAvailable          225 non-null    int64  
 8   properties.closeTime               225 non-null    object 
 9   properties.docksAvailable          225 non-null    int64  
 10  properties.eventEnd                0 non-null      object 
 11  properties.eventStart              0 non-null      object 

In [5]:
stations_df = stations_df[
    [
        "properties.latitude",
        "properties.longitude",
        "properties.addressZipCode",
        "properties.totalDocks",
        "properties.isEventBased",
        "properties.isVirtual",
        "properties.isVisible",
        "properties.kioskId",
        "properties.openTime",
        "properties.closeTime",
        "properties.hasGeofence"
    ]
]
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   properties.latitude        225 non-null    float64
 1   properties.longitude       225 non-null    float64
 2   properties.addressZipCode  225 non-null    object 
 3   properties.totalDocks      225 non-null    int64  
 4   properties.isEventBased    225 non-null    bool   
 5   properties.isVirtual       225 non-null    bool   
 6   properties.isVisible       225 non-null    bool   
 7   properties.kioskId         225 non-null    int64  
 8   properties.openTime        225 non-null    object 
 9   properties.closeTime       225 non-null    object 
 10  properties.hasGeofence     225 non-null    bool   
dtypes: bool(4), float64(2), int64(2), object(3)
memory usage: 13.3+ KB


In [6]:
stations_df.columns = stations_df.columns.str.replace('properties.', '', regex=False)
stations_df.head(2)

,latitude,longitude,addressZipCode,totalDocks,isEventBased,isVirtual,isVisible,kioskId,openTime,closeTime,hasGeofence
0,34.04850,-118.25854,90017,31,False,False,False,3005,05:45:00,05:39:00,False
1,34.04554,-118.25667,90014,31,False,False,False,3006,05:45:00,05:39:00,False


In [7]:
stations_df['openTimeTemp'] = pd.to_datetime(stations_df['openTime'])
stations_df['openTimeHour'] = stations_df['openTimeTemp'].dt.hour
stations_df['openTimeMinute'] = stations_df['openTimeTemp'].dt.minute
stations_df = stations_df.drop(columns=['openTimeTemp'])

stations_df['closeTimeTemp'] = pd.to_datetime(stations_df['closeTime'])
stations_df['closeTimeHour'] = stations_df['closeTimeTemp'].dt.hour
stations_df['closeTimeMinute'] = stations_df['closeTimeTemp'].dt.minute
stations_df = stations_df.drop(columns=['closeTimeTemp'])


In [8]:
stations_df.to_pickle('../00_data/stations.pkl')